In [2]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [4]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [5]:
def extract_max(pitches,magnitudes, shape):
    new_pitches = []
    new_magnitudes = []
    for i in range(0, shape[1]):
        new_pitches.append(np.max(pitches[:,i]))
        new_magnitudes.append(np.max(magnitudes[:,i]))
    return (new_pitches,new_magnitudes)


In [6]:
def smooth(x,window_len=11,window='hanning'):
        if window_len<3:
                return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                raise ValueError
        s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=np.ones(window_len,'d')
        else:
                w=eval('np.'+window+'(window_len)')
        y=np.convolve(w/w.sum(),s,mode='same')
        return y[window_len:-window_len+1]

In [7]:
def set_variables(sample_f,duration,window_time,fmin,fmax,overlap):
    total_samples = sample_f * duration
    #There are sample_f/1000 samples / ms
    #windowsize = number of samples in one window
    window_size = sample_f/1000 * window_time
    hop_length = total_samples / window_size
    #Calculate number of windows needed
    needed_nb_windows = total_samples / (window_size - overlap)
    n_fft = needed_nb_windows * 2.0
    return total_samples, window_size, needed_nb_windows, n_fft, hop_length

In [8]:
def analyse(y,sr,n_fft,hop_length,fmin,fmax):
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, S=None, n_fft= n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, threshold=0.75)
    shape = np.shape(pitches)
    #nb_samples = total_samples / hop_length
    nb_samples = shape[0]
    #nb_windows = n_fft / 2
    nb_windows = shape[1]
    pitches,magnitudes = extract_max(pitches, magnitudes, shape)

    pitches1 = smooth(pitches,window_len=5)
    pitches2 = smooth(pitches,window_len=20)
    pitches3 = smooth(pitches,window_len=30)
    pitches4 = smooth(pitches,window_len=40)
    
#    plot(pitches1, 'pitches1')
#    plot(pitches2, 'pitches2')
#    plot(pitches3, 'pitches3')
#    plot(pitches4, 'pitches4')
#   plot(magnitudes, 'magnitudes')
#    plot( y, 'audio')
    return pitches1


In [10]:
perf_id = '360_26934219'
path = 'audio/'+perf_id+'.m4a'
path2 = '188let_it_go.m4a'
y, sr=librosa.load(path)
    #we want a sample frequency of 16 000
sample_f = 22050
    #The duration of the voice sample
duration = librosa.core.get_duration(y,sr)

    #We want a windowsize of 30 ms
window_time = 50
fmin = 60
fmax = 700
    #We want an overlap of 10 ms
overlap = 10

total_samples, window_size, needed_nb_windows, n_fft, hop_length = set_variables(sample_f, duration, window_time, fmin, fmax, overlap)


    # y = audio time series
    # sr = sampling rate of y
y, sr = librosa.load(path, sr=sample_f, duration=duration)
#y1, sr1 = librosa.load(path2, sr=sample_f, duration=27)
n_fft =2048
n_fft = int(n_fft)
hop_length = int(hop_length)


pitches2 = analyse(y, sr, n_fft, hop_length, fmin, fmax)
len(pitches2)

1103

In [11]:
pitches2[40:140]

array([ 14.87850189,  29.75700378,  14.87850189,  14.98962116,
        29.97924232,  14.98962116,  15.23709011,  45.33577633,
        44.96028233,  14.86159611,  14.97275162,  29.94550323,
        14.97275162,  16.05009079,  32.10018158,  16.05009079,
         0.        ,  15.5216217 ,  31.04324341,  15.5216217 ,
         0.        ,   0.        , 128.56929016, 326.09477997,
       320.03487015, 237.60416412, 233.11200333, 284.80031586,
       331.7269783 , 276.84307098, 169.16197586,  57.26278687,
       104.36038971, 208.72077942, 160.43013382, 275.26302338,
       431.12801361, 359.05503082, 342.60329056, 368.6713562 ,
       286.52401352, 231.82398224, 325.30167007, 397.84283447,
       323.67214203, 326.65414429, 375.58339691, 291.95488739,
       221.77355957, 230.1156311 , 232.17298889, 257.91251373,
       334.59033966, 383.12722778, 389.90140533, 397.93258667,
       301.95547485, 100.92282104,   0.        ,   0.        ,
         0.        ,   0.        ,  15.47386742, 204.96

In [20]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if float(value) > val] 

In [23]:
pitches3 = remove_values_from_list(pitches3, 60)
len(pitches3)

699

In [24]:
df=pd.read_csv('perfs.csv')
df.head()

,plyrid,perf_key,songid,gender,birthyear,lovetot,zone
0,360,360_35435057,_hey_there_delilah,M,1988.0,3.0,EN
1,360,360_26934219,_closing_time,M,1988.0,2.0,EN
2,360,360_35435961,_burn,M,1988.0,2.0,EN
3,360,360_36313263,_burn,M,1988.0,2.0,EN
4,360,360_35352936,_as_long_as_you_love_me,M,1988.0,2.0,EN


In [25]:
arrang_list = df['songid'].value_counts()
df_arrang = pd.DataFrame(arrang_list)
df_arrang = df_arrang.reset_index()
arrang = list(df_arrang['index'])
arrang

['_let_it_go',
 '_stay_rihanna',
 '_say_something_great_big',
 '_cups_pitch_perfect',
 '_when_i_was_your_man',
 '_the_climb',
 '_my_heart_will_go_on',
 '_let_her_go',
 '_i_wont_give_up',
 '_hallelujah',
 '_apologize',
 '_part_of_your_world',
 '_demons',
 '_because_of_you',
 '_killing_me_softly',
 '_counting_stars',
 '_summertime_sadness',
 '_grenade',
 '_beautiful',
 '_a_thousand_miles',
 '_over_the_rainbow',
 '_wind_beneath_my_wings',
 '_hey_there_delilah',
 '_colors_of_the_wind',
 '_adore_you',
 '_i_dreamed_a_dream',
 '_hero_mariah_carey',
 '_crazy_patsy',
 '_radioactive',
 '_im_yours',
 '_human',
 '_the_a_team',
 '_call_me_maybe',
 '_the_rose',
 '_happy',
 '_amazing_grace_elvis',
 '_pompeii',
 '_neon_lights',
 '_bubbly',
 '_dont_know_why',
 '_if_i_aint_got_you',
 '_you_are_not_alone',
 '_stand_by_me',
 '_before_he_cheats',
 '_aint_no_sunshine',
 '_alone_heart',
 '_at_last',
 '_you_raise_me_up',
 '_just_the_way_you_are_bm',
 '_on_my_own',
 '_twinkle_twinkle',
 '_clarity',
 '_kiss_me_

In [26]:
array_perf = []
for arr in arrang:
    new_df = df.loc[df['songid']==arr]
    new_df = new_df.head(21)
    new_list = list(new_df['perf_key'])
    for li in new_list:
        array_perf.append(li)
    print(new_list)


['67982109_31286272', '68366569_39548661', '68366569_39548632', '68366569_40173934', '68661233_31871812', '68700363_40360494', '70351731_35425066', '70351731_42103937', '70366197_41259492', '70366197_41433490', '70366197_41629933', '70366197_43227220', '70366197_41804302', '70366197_41304818', '70366197_41301672', '70366197_41628859', '70903164_30829647', '70903164_30830264', '70903164_30832307', '70903164_45081601', '70903164_42999512']
['69411204_36121737', '70167169_37897098', '72214345_43074484', '72745985_32491677', '73260958_40652595', '74210056_24195427', '74363136_40656583', '74530905_40128867', '75797746_30902683', '76571844_24429366', '76779448_23931819', '76779448_23930731', '76852358_37891505', '76852358_42266122', '76852358_22834987', '76930143_28104359', '77031751_25588774', '77300542_37438599', '78673792_40252316', '78829692_28153153', '78829692_25585851']
['68307032_37623498', '68500763_35809367', '68500763_30671765', '69164761_32708165', '69411204_33391558', '69443616_

['69668503_24049411', '74487811_47778094', '76535424_38622568', '79605110_43809028', '80504974_34681504', '83417526_41985534', '84467294_31566069', '85644296_34707253', '86217510_40560848', '87881815_36793801', '88196629_37624288', '89040487_33919393', '90501328_30980942', '91635849_34130879', '91645767_36694105', '91962472_30861758', '95521146_22989877', '96081676_22336883', '96081676_22952330', '98732900_22347112', '101713925_44730548']
['70940743_24007423', '70940743_23355699', '70940743_23079319', '74471372_25674207', '77385484_43438875', '77385484_43166284', '77385484_43167344', '77385484_43777038', '77385484_43170897', '77385484_43772275', '77385484_48745786', '77385484_48747443', '78397516_41054289', '78960019_40628557', '80115382_37223152', '80115382_24457854', '82212517_36770272', '85090690_22781943', '85644296_35107957', '87696931_35037853', '94800857_34387066']
['74313351_46970274', '76868786_33970750', '78611032_50325164', '80290811_24254840', '84032291_41749899', '93379933

['68500763_34003146', '70945242_31185406', '78231051_28266988', '88751158_45550001', '91510715_42640557', '92012499_35259407', '92012499_39673160', '96028167_25463077', '96081676_22951803', '97890120_41416032', '99211008_33618614', '108234504_35576648', '108652764_28279001', '110642165_44768164', '111652826_29430623', '111712578_48303159', '114448765_41924305', '114448765_42412605', '116953806_26404444', '118268520_26032443', '122201194_24949053']
['70943421_39590656', '75607754_34000159', '75885060_30372467', '75885060_30372953', '84150160_43319890', '85088130_25697763', '89430362_41293939', '89430362_40434602', '94474797_34250018', '100541863_26220509', '102495711_41206290', '102495711_40462793', '102495711_40547824', '102495711_40603718', '109654776_30020748', '111227011_33425907', '115157290_30991078', '115480091_26861773', '120232156_39931286', '123994528_21310716', '125721227_46266225']
['360_21740786', '70943421_28854779', '77729004_26145586', '77729004_29585917', '79823079_2910

['74313351_47344461', '76694768_49451277', '81142361_46754651', '84955650_46712037', '87463607_46503119', '89130741_50846107', '89419746_48560951', '90470919_46773458', '93704701_48519287', '94641679_47040640', '103222576_46563456', '103233388_46779989', '104069446_46546707', '104069446_46524665', '104902673_47540610', '111237677_50803299', '116082404_47281429', '116082404_47352580', '119956279_50360618', '120505161_46892931', '120505161_46604197']
['70315264_29246771', '80504974_38671620', '82339301_29242257', '86818415_29478238', '88062126_47317998', '95516523_37660510', '96469350_29767151', '98595226_30418792', '103301910_37510088', '111237677_38117841', '114922272_46634044', '115797422_38969433', '117773275_35524000', '118149937_48804408', '118149937_43396984', '118452064_29383338', '118461392_32736578', '118461392_32934027', '118851568_31281641', '118851568_41500536', '120535815_38620495']
['74471372_28042025', '74471372_25775599', '74471372_26561450', '84496441_49751645', '937047

In [27]:
df_v = df.loc[df['perf_key'].isin(array_perf)]
df_v2 = df_v.loc[df['songid'].isin(arrang[:10])]
len(df_v2)

210

In [86]:
sample_f = 22050
    #We want a windowsize of 30 ms
window_time = 50
fmin = 60
fmax = 700
    #We want an overlap of 10 ms
overlap = 10


vocals = []
ind = 0
for perf in df_v2['perf_key']:
    print(ind)
    ind=ind+1
    y, sr = librosa.load('audio/'+perf+'.m4a')
    duration = librosa.core.get_duration(y,sr)
    total_samples, window_size, needed_nb_windows, n_fft, hop_length = set_variables(sample_f, duration, window_time, fmin, fmax, overlap)
    n_fft =2048
    n_fft = int(n_fft)
    hop_length = int(hop_length)
    pitches2 = analyse(y, sr, n_fft, hop_length, fmin, fmax)
    pitches = list(pitches2)
    pitches = remove_values_from_list(pitches, 60)
    pitches = pitches[200:600]
    pitches.append(perf)
    vocals.append(pitches)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


In [87]:
len(vocals)

210

In [59]:
y,sr = librosa.load('audio/138785282_33276605.m4a')

/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


FileNotFoundError: [Errno 2] No such file or directory: 'audio/138785282_33276605.m4a'

In [88]:
vocals_df = pd.DataFrame(vocals)

In [89]:
vocals_df.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,314.228264,288.298237,250.616508,379.952930,453.860657,381.749313,333.868675,342.141830,340.009842,342.491844,...,408.054909,420.664131,400.471786,398.596619,368.342598,332.421089,326.145355,332.337021,413.770775,67982109_30565735
1,695.910324,694.061905,690.892456,645.916595,517.017036,302.136368,86.328697,86.603851,237.671280,313.588089,...,298.109657,99.013954,103.023132,293.971695,437.347061,499.155434,449.804100,299.810463,102.763298,67982109_31286272
2,472.623856,284.764748,202.701649,256.258209,366.989986,428.514679,481.160271,558.678848,503.011444,421.491470,...,536.112366,412.691116,350.424644,427.061272,489.844025,493.466156,425.225185,352.534439,355.876492,68307032_37628522
3,439.301460,530.879562,486.802872,330.016731,223.844101,136.388725,140.514088,184.459965,137.079998,89.983707,...,376.547455,486.004478,570.530594,502.069168,505.238548,611.394531,662.646698,540.686058,388.409103,68307032_39376090
4,277.692635,239.075119,207.528107,210.685852,212.860340,272.775261,437.210102,479.397758,329.794388,268.237900,...,218.773849,210.191917,223.447506,238.804775,238.726997,238.358547,238.059097,235.708878,175.441467,68307032_37606632


In [90]:
vocals_df[400].unique
vocals_df = vocals_df.rename(columns = {400:'perf_key'})

In [91]:
df=df[['songid','perf_key']]
df.head()

,songid,perf_key
0,_hey_there_delilah,360_35435057
1,_closing_time,360_26934219
2,_burn,360_35435961
3,_burn,360_36313263
4,_as_long_as_you_love_me,360_35352936


In [92]:

new_df=pd.merge(vocals_df, df,on='perf_key')
new_df = new_df.dropna(how='any')

In [93]:
new_df['songid'].value_counts()

_i_wont_give_up             21
_the_climb                  21
_let_her_go                 21
_hallelujah                 21
_cups_pitch_perfect         21
_stay_rihanna               21
_when_i_was_your_man        21
_let_it_go                  21
_my_heart_will_go_on        21
_say_something_great_big    21
Name: songid, dtype: int64

In [94]:
part_array = list(new_df['perf_key'])
part_array

['67982109_30565735',
 '67982109_31286272',
 '68307032_37628522',
 '68307032_39376090',
 '68307032_37606632',
 '68307032_37623498',
 '68366569_28940318',
 '68366569_39548661',
 '68366569_39548632',
 '68366569_40173934',
 '68500763_35809367',
 '68500763_30671765',
 '68500763_25485652',
 '68661233_31871812',
 '68661233_45811856',
 '68700363_39526345',
 '68700363_40360494',
 '68722250_33705744',
 '68722250_33726646',
 '68846173_38545914',
 '68874527_42510598',
 '68874527_44187555',
 '68874527_42625797',
 '69164761_32708165',
 '69164761_26657637',
 '69251376_31194361',
 '69411204_33391558',
 '69411204_36121737',
 '69443616_41218829',
 '69443616_25813589',
 '69668503_37079635',
 '69766860_28608218',
 '69902249_47794385',
 '69902249_42889092',
 '69902249_47403858',
 '69902249_46736053',
 '69902249_42737221',
 '69902249_32979924',
 '69902249_31893791',
 '69902249_42751940',
 '70167169_49286652',
 '70167169_35653426',
 '70167169_35475188',
 '70167169_37897098',
 '70167169_34609285',
 '70315264

In [95]:
new_df = new_df.dropna(how='any')
new_df.head()

,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,perf_key,songid
0,314.228264,288.298237,250.616508,379.952930,453.860657,381.749313,333.868675,342.141830,340.009842,342.491844,...,420.664131,400.471786,398.596619,368.342598,332.421089,326.145355,332.337021,413.770775,67982109_30565735,_hallelujah
1,695.910324,694.061905,690.892456,645.916595,517.017036,302.136368,86.328697,86.603851,237.671280,313.588089,...,99.013954,103.023132,293.971695,437.347061,499.155434,449.804100,299.810463,102.763298,67982109_31286272,_let_it_go
2,472.623856,284.764748,202.701649,256.258209,366.989986,428.514679,481.160271,558.678848,503.011444,421.491470,...,412.691116,350.424644,427.061272,489.844025,493.466156,425.225185,352.534439,355.876492,68307032_37628522,_let_her_go
3,439.301460,530.879562,486.802872,330.016731,223.844101,136.388725,140.514088,184.459965,137.079998,89.983707,...,486.004478,570.530594,502.069168,505.238548,611.394531,662.646698,540.686058,388.409103,68307032_39376090,_cups_pitch_perfect
4,277.692635,239.075119,207.528107,210.685852,212.860340,272.775261,437.210102,479.397758,329.794388,268.237900,...,210.191917,223.447506,238.804775,238.726997,238.358547,238.059097,235.708878,175.441467,68307032_37606632,_let_her_go


In [96]:
new_df['songid'].unique()

array(['_hallelujah', '_let_it_go', '_let_her_go', '_cups_pitch_perfect',
       '_say_something_great_big', '_i_wont_give_up',
       '_my_heart_will_go_on', '_when_i_was_your_man', '_the_climb',
       '_stay_rihanna'], dtype=object)

In [97]:
X = new_df.drop('perf_key', axis=1).drop('songid', axis =1)
y = new_df['songid']
print(X.shape, y.shape)

(210, 400) (210,)


In [98]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [99]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [100]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [101]:
y_train_categorical = to_categorical(encoded_y_train, num_classes = 10)
y_test_categorical = to_categorical(encoded_y_test,num_classes = 10)

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=400))
#model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

In [107]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               40100     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1010      
Total params: 41,110
Trainable params: 41,110
Non-trainable params: 0
_________________________________________________________________


In [108]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 157 samples
Epoch 1/100
157/157 - 0s - loss: 2.4798 - accuracy: 0.0701
Epoch 2/100
157/157 - 0s - loss: 2.3018 - accuracy: 0.1083
Epoch 3/100
157/157 - 0s - loss: 2.2194 - accuracy: 0.1975
Epoch 4/100
157/157 - 0s - loss: 2.1814 - accuracy: 0.2420
Epoch 5/100
157/157 - 0s - loss: 2.1148 - accuracy: 0.2994
Epoch 6/100
157/157 - 0s - loss: 2.0686 - accuracy: 0.3121
Epoch 7/100
157/157 - 0s - loss: 1.9976 - accuracy: 0.4713
Epoch 8/100
157/157 - 0s - loss: 1.9553 - accuracy: 0.5032
Epoch 9/100
157/157 - 0s - loss: 1.9025 - accuracy: 0.5541
Epoch 10/100
157/157 - 0s - loss: 1.8430 - accuracy: 0.5605
Epoch 11/100
157/157 - 0s - loss: 1.7890 - accuracy: 0.6433
Epoch 12/100
157/157 - 0s - loss: 1.7541 - accuracy: 0.5796
Epoch 13/100
157/157 - 0s - loss: 1.6803 - accuracy: 0.7325
Epoch 14/100
157/157 - 0s - loss: 1.6287 - accuracy: 0.7197
Epoch 15/100
157/157 - 0s - loss: 1.5667 - accuracy: 0.7134
Epoch 16/100
157/157 - 0s - loss: 1.5140 - accuracy: 0.7643
Epoch 17/100
157/157 - 0s - 

In [109]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

53/1 - 0s - loss: 1.9315 - accuracy: 0.5283
Normal Neural Network - Loss: 1.8509579604526736, Accuracy: 0.5283018946647644


In [110]:
y2, sr = librosa.load('188let_it_go.m4a')
y2

/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


array([-1.2140491e-05,  1.4367494e-05, -1.7592216e-05, ...,
       -4.8620203e-03, -3.9158552e-03,  0.0000000e+00], dtype=float32)

In [114]:
duration = librosa.core.get_duration(y2,sr)
total_samples, window_size, needed_nb_windows, n_fft, hop_length = set_variables(sample_f, duration, window_time, fmin, fmax, overlap)
n_fft =2048
n_fft = int(n_fft)
hop_length = int(hop_length)
pitches2 = analyse(y2, sr, n_fft, hop_length, fmin, fmax)
pitches = list(pitches2)[200:600]

In [115]:
test_df = pd.DataFrame(pitches)
test_df = test_df.transpose()

In [116]:
X_test_scaled1 = X_scaler.transform(test_df)

In [117]:
encoded_predictions1 = model.predict_classes(X_test_scaled1)
prediction_labels1 = label_encoder.inverse_transform(encoded_predictions1)

In [118]:
prediction_labels1

array(['_say_something_great_big'], dtype=object)

In [119]:
ynew = model.predict_proba(X_test_scaled1)
#prediction_labelsy = label_encoder.inverse_transform(ynew)
#prediction_labelsy
ynew

array([[9.50634573e-03, 9.42161679e-03, 1.48487336e-04, 3.59720638e-04,
        1.96327548e-02, 1.11735106e-01, 4.45093006e-01, 3.80764782e-01,
        2.02966928e-02, 3.04146553e-03]], dtype=float32)

In [59]:
encoded_predictions1

array([6])

In [61]:
prediction_labelsy = label_encoder.inverse_transform(np.arange(ynew))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [71]:
label_encoder.inverse_transform(([5]))

array(['_my_heart_will_go_on'], dtype=object)